<a href="https://colab.research.google.com/github/Malanidhruv/Python-projects/blob/main/Stocks%20Research/Trades%20for%20tomorrow/Fno_considering_3_15.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import numpy as np
from scipy.signal import argrelextrema
import pandas as pd
from pya3 import Aliceblue
from datetime import datetime
from nsetools import Nse

def get_user_credentials():
    user_id = input("Enter your user ID: ")
    api_key = input("Enter your API key: ")
    return user_id, api_key

def initialize_alice():
    user_id, api_key = get_user_credentials()
    alice = Aliceblue(user_id=user_id, api_key=api_key)
    alice.get_session_id()
    return alice

def analyze_stock(alice, token):
    instrument = alice.get_instrument_by_token('NSE', token)

    from_datetime = datetime(2022, 1, 1)  # Start from a longer time frame
    to_datetime = datetime.now()  # To now
    interval = "D"  # ["1", "D"]

    historical_data = alice.get_historical(instrument, from_datetime, to_datetime, interval)
    df = pd.DataFrame(historical_data)

    # Find local maxima and minima
    close_prices = df['close'].values
    local_maxima_indices = argrelextrema(close_prices, np.greater)[0]
    local_minima_indices = argrelextrema(close_prices, np.less)[0]

    # Global maxima and minima
    global_maxima_index = np.argmax(close_prices)
    global_minima_index = np.argmin(close_prices)

    # Define the current price (example: last close price)
    current_price = df['close'].iloc[-1]

    # Define threshold (e.g., within 5% of the extrema)
    threshold = 0.05

    # Determine if current price is near any local or global minima or maxima
    def is_near_extreme(current_price, extreme_price, threshold):
        return abs((current_price - extreme_price) / extreme_price) < threshold

    # Check proximity to local minima
    near_local_minima = any(is_near_extreme(current_price, close_prices[i], threshold) for i in local_minima_indices)

    # Check proximity to global minima
    near_global_minima = is_near_extreme(current_price, close_prices[global_minima_index], threshold)

    # Calculate the moving average of the volume
    df['volume_ma'] = df['volume'].rolling(window=20).mean()

    # Volume threshold (e.g., 1.5 times the average volume)
    volume_threshold = 1.5 * df['volume'].mean()

    # Enhanced signal determination using volume
    if (near_local_minima or near_global_minima) and (current_price > (close_prices[local_minima_indices[np.argmin(abs(close_prices[local_minima_indices] - current_price))]] if near_local_minima else close_prices[global_minima_index])):
        signal = "BUY"
        if near_local_minima:
            nearest_local_minima_index = local_minima_indices[np.argmin(abs(close_prices[local_minima_indices] - current_price))]
            support_level = close_prices[nearest_local_minima_index]
            support_volume = df.loc[nearest_local_minima_index, 'volume']
        else:
            support_level = close_prices[global_minima_index]
            support_volume = df.loc[global_minima_index, 'volume']
        if support_volume < volume_threshold:
            signal = None  # Downgrade to None if volume is not confirming
    else:
        signal = None

    return token, current_price, signal, support_level if signal == "BUY" else None

def print_signals(buy_signals):
    print("BUY Signals:")
    print(f"{'Name':<15} {'Token':<10} {'Current Price':<15} {'Support Level':<15}")
    print('-' * 55)
    for signal in buy_signals:
        print(f"{signal['Name']:<15} {signal['Token']:<10} {signal['Current Price']:<15.2f} {signal['Support Level']:<15.2f}")
    print("--------------------------------------------------")

def get_stock_name(alice, token):
    instrument = alice.get_instrument_by_token('NSE', token)
    return instrument.name

# Initialize Aliceblue object and get credentials
alice = initialize_alice()

# List of tokens to analyze
tokens = [
    11532, 4306, 467, 2885, 1394, 10999, 547, 1232, 1922, 526, 17818, 21808, 17963, 1660, 11536, 15083, 3456, 11483, 13538, 236, 20374, 10604, 25, 14977, 4963, 5258, 1594, 10940, 3351, 3787, 11630, 3432, 317, 7229, 3045, 1348, 16675, 881, 3506, 11723, 2475, 1333, 910, 5900, 694, 2031, 157, 1363, 16669, 3499
]

# Analyze each stock
buy_signals = []

for token in tokens:
    result = analyze_stock(alice, token)
    if result[2] == "BUY":
        buy_signals.append((result[0], result[1], result[3]))

# Adding stock names to the signals
for i in range(len(buy_signals)):
    buy_signals[i] = {
        'Token': buy_signals[i][0],
        'Current Price': buy_signals[i][1],
        'Support Level': buy_signals[i][2],
        'Name': get_stock_name(alice, buy_signals[i][0])
    }

# Print signals
print_signals(buy_signals)


Enter your user ID: 1141826
Enter your API key: 73Beb7IO1hC2iZJ38bkeLHMQB3jilZpn5J3qpKanQahoV2teJzE5LQtG7NM1kX5EsjkDsFKYK0T5P9XPcXcRf1qpZSJffZscIe9L5QVqQnhThnLcMmPTFxdYK0xh080g
NOTE: Today's contract master file will be updated after 08:00 AM. Before 08:00 AM previous day contract file be downloaded.
BUY Signals:
Name            Token      Current Price   Support Level  
-------------------------------------------------------
SHRIRAMFIN-EQ   4306       585.10          559.55         
NESTLEIND-EQ    17963      2214.95         2214.45        
INDUSINDBK-EQ   5258       1043.75         1041.85        
--------------------------------------------------


In [2]:
%pip install pya3 nsetools

In [5]:
import numpy as np
from scipy.signal import argrelextrema
import pandas as pd
from pya3 import Aliceblue
from datetime import datetime, timedelta
from sklearn.preprocessing import MinMaxScaler

def analyze_stock(alice, token):
    # Get instrument and historical data (last 1 year)
    instrument = alice.get_instrument_by_token('NSE', token)
    from_date = datetime.now() - timedelta(days=365)
    historical_data = alice.get_historical(instrument, from_date, datetime.now(), "D")
    df = pd.DataFrame(historical_data).dropna()

    if len(df) < 30:  # Ensure sufficient data
        return None

    # Enhanced Support/Resistance Detection
    close_prices = df['close'].values
    scaler = MinMaxScaler()
    normalized_prices = scaler.fit_transform(close_prices.reshape(-1, 1)).flatten()

    # Dynamic window size based on data length
    window_size = max(int(len(df) * 0.1), 5)
    local_min = argrelextrema(normalized_prices, np.less_equal, order=window_size)[0]
    local_max = argrelextrema(normalized_prices, np.greater_equal, order=window_size)[0]

    # Filter significant support levels (last 3 months)
    recent_min = [m for m in local_min if m > len(df)-60]
    valid_supports = []
    for m in recent_min:
        support_price = close_prices[m]
        # Check if price is above support and within 15% range
        if (close_prices[-1] > support_price * 1.02 and
            close_prices[-1] < support_price * 1.15):
            # Volume validation (current volume > support period volume)
            if df['volume'].iloc[-1] > df['volume'].iloc[m] * 1.2:
                valid_supports.append((m, support_price))

    if not valid_supports:
        return None

    # Find strongest support (most touches in cluster)
    support_clusters = []
    for sup in valid_supports:
        found = False
        for cluster in support_clusters:
            if abs(sup[1] - cluster['price']) < np.std(close_prices)*0.5:
                cluster['count'] += 1
                found = True
                break
        if not found:
            support_clusters.append({'price': sup[1], 'count': 1})

    if not support_clusters:
        return None

    best_support = max(support_clusters, key=lambda x: x['count'])
    current_price = close_prices[-1]

    # Additional technical confirmation
    ma_50 = df['close'].rolling(50).mean().iloc[-1]
    ma_200 = df['close'].rolling(200).mean().iloc[-1]
    rsi = compute_rsi(df['close'], 14)

    # Final signal criteria
    if (current_price > ma_50 > ma_200 and
        best_support['count'] >= 2 and
        rsi < 60 and
        (current_price - best_support['price']) > np.std(close_prices[-30:])*0.5):
        return {
            'Token': token,
            'Name': instrument.name,
            'Current Price': current_price,
            'Support Level': best_support['price'],
            'Support Strength': best_support['count'],
            'Distance %': round((current_price/best_support['price']-1)*100, 2)
        }
    return None

def compute_rsi(prices, window):
    delta = prices.diff()
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)

    avg_gain = gain.rolling(window).mean()
    avg_loss = loss.rolling(window).mean()

    rs = avg_gain / avg_loss
    return 100 - (100 / (1 + rs)).iloc[-1]

# ... (Keep get_user_credentials and initialize_alice same as before)

def print_signals(signals):
    print("Top BUY Signals:")
    print(f"{'Stock':<20} {'Price':<10} {'Support':<10} {'Strength':<10} {'Distance%':<10}")
    for s in signals:
        print(f"{s['Name'][:18]:<20} {s['Current Price']:<10.1f} {s['Support Level']:<10.1f} "
              f"{s['Support Strength']:<10} {s['Distance %']:<10.1f}")

# Main execution
alice = initialize_alice()
tokens = [
    11532, 4306, 467, 2885, 1394, 10999, 547, 1232, 1922, 526, 17818, 21808, 17963, 1660, 11536, 15083, 3456, 11483, 13538, 236, 20374, 10604, 25, 14977, 4963, 5258, 1594, 10940, 3351, 3787, 11630, 3432, 317, 7229, 3045, 1348, 16675, 881, 3506, 11723, 2475, 1333, 910, 5900, 694, 2031, 157, 1363, 16669, 3499
]

signals = []
for token in tokens:
    try:
        result = analyze_stock(alice, token)
        if result:
            signals.append(result)
    except Exception as e:
        print(f"Error processing {token}: {str(e)}")

# Select top 10 stocks with strongest support and reasonable distance
top_signals = sorted(signals,
                    key=lambda x: (-x['Support Strength'], -x['Distance %']))[:10]

print_signals(top_signals)

Enter your user ID: 1141826
Enter your API key: 73Beb7IO1hC2iZJ38bkeLHMQB3jilZpn5J3qpKanQahoV2teJzE5LQtG7NM1kX5EsjkDsFKYK0T5P9XPcXcRf1qpZSJffZscIe9L5QVqQnhThnLcMmPTFxdYK0xh080g
Top BUY Signals:
Stock                Price      Support    Strength   Distance% 


In [8]:
import numpy as np
from scipy.signal import argrelextrema
import pandas as pd
from pya3 import Aliceblue
from datetime import datetime, timedelta
from sklearn.preprocessing import MinMaxScaler
import time

def get_user_credentials():
    return "1141826", "73Beb7IO1hC2iZJ38bkeLHMQB3jilZpn5J3qpKanQahoV2teJzE5LQtG7NM1kX5EsjkDsFKYK0T5P9XPcXcRf1qpZSJffZscIe9L5QVqQnhThnLcMmPTFxdYK0xh080g"  # Replace with actual credentials

def initialize_alice():
    user_id, api_key = get_user_credentials()
    alice = Aliceblue(user_id=user_id, api_key=api_key)
    print("Session ID:", alice.get_session_id())
    return alice

def compute_rsi(prices, window=14):
    delta = prices.diff()
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)

    avg_gain = gain.rolling(window).mean()
    avg_loss = loss.rolling(window).mean()

    rs = avg_gain / avg_loss
    return 100 - (100 / (1 + rs)).iloc[-1]

def analyze_stock(alice, token):
    try:
        print(f"\nAnalyzing token {token}...")
        instrument = alice.get_instrument_by_token('NSE', token)

        # Get 2 years of daily data
        from_date = datetime.now() - timedelta(days=730)
        historical_data = alice.get_historical(instrument, from_date, datetime.now(), "D")
        df = pd.DataFrame(historical_data).dropna()

        print(f"Data points: {len(df)}")
        if len(df) < 100:
            print("Insufficient data")
            return None

        # Basic technical indicators
        df['50_EMA'] = df['close'].ewm(span=50).mean()
        df['200_EMA'] = df['close'].ewm(span=200).mean()
        rsi = compute_rsi(df['close'])

        # Support/resistance detection
        close_prices = df['close'].values
        scaler = MinMaxScaler()
        normalized_prices = scaler.fit_transform(close_prices.reshape(-1, 1)).flatten()

        # Dynamic window size
        window_size = max(int(len(df)*0.05), 5)
        local_min = argrelextrema(normalized_prices, np.less_equal, order=window_size)[0]

        # Find significant support levels (last 6 months)
        valid_supports = []
        for m in local_min:
            if m < len(df)-126:  # Older than 6 months
                continue

            support_price = close_prices[m]
            current_price = close_prices[-1]

            # Price between 5% to 20% above support
            if 1.05 <= (current_price / support_price) <= 1.20:
                # Volume check (current volume > support period volume)
                if df['volume'].iloc[-1] > df['volume'].iloc[m] * 0.8:
                    valid_supports.append({
                        'price': support_price,
                        'date': df.index[m],
                        'touches': 1
                    })

        print(f"Found {len(valid_supports)} potential supports")

        if not valid_supports:
            return None

        # Cluster nearby supports
        support_clusters = []
        tolerance = np.std(close_prices)*0.3
        for sup in valid_supports:
            found = False
            for cluster in support_clusters:
                if abs(sup['price'] - cluster['price']) <= tolerance:
                    cluster['count'] += 1
                    found = True
                    break
            if not found:
                support_clusters.append({
                    'price': sup['price'],
                    'count': 1,
                    'dates': [sup['date']]
                })

        if not support_clusters:
            return None

        # Select best support cluster
        best_cluster = max(support_clusters, key=lambda x: x['count'])
        print(f"Best support: {best_cluster['price']} (Count: {best_cluster['count']})")

        # Trend filter
        if df['50_EMA'].iloc[-1] < df['200_EMA'].iloc[-1]:
            print("Failing trend filter (50EMA < 200EMA)")
            return None

        # Momentum filter
        if rsi > 65:
            print(f"RSI too high: {rsi:.1f}")
            return None

        # Risk management check
        current_price = close_prices[-1]
        distance_pct = (current_price / best_cluster['price'] - 1) * 100

        print(f"Current: {current_price}, Support: {best_cluster['price']}, Distance: {distance_pct:.1f}%")

        return {
            'Token': token,
            'Name': instrument.name.split('-')[0].strip(),
            'Price': current_price,
            'Support': best_cluster['price'],
            'Strength': best_cluster['count'],
            'Distance%': distance_pct,
            'RSI': rsi,
            'Trend': 'Bullish' if df['50_EMA'].iloc[-1] > df['200_EMA'].iloc[-1] else 'Bearish'
        }

    except Exception as e:
        print(f"Error analyzing {token}: {str(e)}")
        return None

def main():
    alice = initialize_alice()

    # Example tokens (Nifty 50 constituents)
    tokens = [
14747, 13197, 23891, 14599, 1247, 7982, 5475, 15347, 6543, 10871, 5054, 13560, 18742, 20110, 3987, 17770, 9768,
               1135, 1162, 20854, 48, 2303, 16783, 10865, 13229, 6286, 11581, 4391, 10097, 10599, 14677, 1267, 11606,
               21195, 13745, 13810, 6848, 17092, 1235, 11971, 11619, 14480, 16705, 11778, 5010, 17875, 10925, 10099, 144,
               23799, 1610, 1181, 25943, 6964, 3963, 13701, 8964, 1, 6125, 14052, 20534, 1232, 592, 20454, 11872, 17685, 22846,
               19731, 13409, 24113, 19356, 11787, 23066, 9874, 14687, 8968, 11162, 23826, 12173, 19211, 19698, 17053, 1448, 2289,
               18653, 8568, 2868, 19322, 13890, 1424, 15883, 1394, 1406, 15161, 2056, 13526, 19813, 9610, 21364, 25844, 12809, 25711,
               2562, 20825, 14203, 11979, 14334, 21120, 18679, 11796, 7376, 13592, 3138, 3417, 1403, 21676, 17939, 4924, 2316, 1336, 7425,
               19205, 467, 1333, 4244, 7229, 1327, 15555, 1375, 4116, 13966, 7457, 9819, 3892, 18154, 701, 1313, 2614, 4598, 1372, 1363, 1360,
               14627, 1455, 9668, 18804, 14712, 1672, 21951, 29666, 27008, 715, 7048, 1348, 15292, 489, 1174, 8529, 12000, 24153, 1614, 1008, 5352,
               17486, 676, 6198, 22947, 11471, 17380, 6017, 11782, 8886, 7331, 25324, 11999, 4898, 1023, 20372, 12032, 24814, 1041, 6579,
               3744, 1038, 23651, 24532, 13710, 9861, 8159, 576, 17006, 13812, 20322, 25162, 27629, 9750, 24265, 15053, 18822, 9756,
               18593, 23748, 18142, 10253, 13424, 11530, 24398, 14450, 9383, 19281, 10074, 13517, 29229, 13423, 23227, 25516,
               4907, 13192, 30323, 25638, 24681, 17084, 958, 22767, 19878, 955, 21154, 913, 3553, 981, 5382, 22463,
               25412, 22717, 23233, 18571, 14868, 1190, 17843, 277, 18816, 1139, 16696, 1127, 24442, 1576, 1289,
               13776, 7862, 11896, 11905, 6600, 17010, 1142, 1145, 2366, 13528, 1570, 5204, 1168, 23590, 17424,
               393, 13739, 17167, 24934, 1415, 7406, 1153, 1186, 17360, 9935, 10905, 2188, 17922, 13658, 8828,
               1085, 12842, 11932, 14304, 11501, 25871, 1073, 14592, 25664, 11573, 24917, 6673, 22517, 6836, 7385,
               21021, 4717, 13750, 25247, 1315, 19675, 3699, 8510, 27, 18018, 25800, 1100, 21399, 4957, 27339,
               14339, 5614, 6944, 14116, 20303, 13337, 25134, 4963, 21770, 18652, 19468, 24385, 18967, 1196, 1782,
               1835, 11208, 1838, 25786, 13457, 1826, 2955, 15160, 19653, 25443, 18755, 13817, 24531, 15136, 15248,
               13125, 29550, 13430, 18708, 13310, 8880, 18220, 13260, 16765, 14908, 4809, 1841, 10296, 21185, 2360,
               12092, 10276, 1811, 1808, 1805, 23462, 11236, 22663, 11763, 31507, 11461, 16927, 23621, 13637, 7109,
               18553, 14435, 11860, 13491, 9038, 1859, 17869, 19020, 1802, 21334, 29619, 15146, 20224, 29962, 22824,
               3637, 2783, 18096, 29592, 7287, 3237, 11723, 11883, 15266, 4157, 425, 1949, 23175, 5359, 25252, 24025,
               25643, 11432, 9296, 8782, 8671, 16147, 12847, 11488, 10577, 14912, 6417, 14972, 10412, 18889, 6818, 21713,
               14423, 13276, 23027, 27225, 6412, 11654, 14929, 25863, 4774, 5865, 13870, 22784, 8054, 11060, 9683, 1814, 5108,
               13381, 28903, 15180, 19025, 20936, 18581, 16639, 19126, 18944, 4847, 19937, 21119, 9570, 9641, 3912, 896, 29195, 13359,
               17781, 27084, 27079, 6444, 6808, 4594, 11647, 1922, 10892, 4951, 19287, 22475, 15124, 16827, 3009, 18751, 15283, 8602,
               14667, 21186, 13270, 18565, 19695, 13275, 3088, 10993, 29025, 11677, 8614, 9116, 25612, 4747, 2048, 11195, 21061, 20556,
               1530, 4888, 14309, 20988, 25505, 14952, 4751, 21062, 25320, 22770, 1597, 1594, 13694, 16588, 11027, 15036, 16249, 29135,
               23036, 5258, 21957, 11605, 4870, 10726, 1521, 28378.
    ]

    signals = []
    for token in tokens:
        result = analyze_stock(alice, token)
        if result:
            signals.append(result)
        time.sleep(1)  # Rate limit

    # Sort by strongest signals
    sorted_signals = sorted(signals,
        key=lambda x: (-x['Strength'], -x['Distance%']))

    print("\nTop 10 Buy Candidates:")
    print(f"{'Stock':<20} {'Price':<10} {'Support':<10} {'Strength':<10} {'Distance%':<10} {'RSI':<6} {'Trend':<10}")
    for s in sorted_signals[:10]:
        print(f"{s['Name'][:18]:<20} {s['Price']:<10.1f} {s['Support']:<10.1f} "
              f"{s['Strength']:<10} {s['Distance%']:<10.1f} {s['RSI']:<6.1f} {s['Trend']:<10}")

if __name__ == "__main__":
    main()

Session ID: {'stat': 'Ok', 'sessionID': 'eyJhbGciOiJSUzI1NiIsInR5cCIgOiAiSldUIiwia2lkIiA6ICIyam9lOFVScGxZU3FTcDB3RDNVemVBQkgxYkpmOE4wSDRDMGVVSWhXUVAwIn0.eyJleHAiOjE3NDUzODY1NzUsImlhdCI6MTc0MDIwMjU3NSwianRpIjoiYTUxODBhMGItMWZjMi00OWVmLWJhYTctOWM3YzE1ZTgzMzU2IiwiaXNzIjoiaHR0cHM6Ly9pZGFhcy5hbGljZWJsdWVvbmxpbmUuY29tL2lkYWFzL3JlYWxtcy9BbGljZUJsdWUiLCJhdWQiOiJhY2NvdW50Iiwic3ViIjoiY2U2ZTQ4MzEtZWQ3My00MGQ2LThlN2YtYWRmZDA4Y2ViMjVjIiwidHlwIjoiQmVhcmVyIiwiYXpwIjoiYWxpY2Uta2IiLCJzaWQiOiIwZGU3Njk3MC03Zjg2LTQ3ZmEtOTVlYS0zN2Y2ZDZiY2QyODQiLCJhbGxvd2VkLW9yaWdpbnMiOlsiaHR0cDovL2xvY2FsaG9zdDozMDAyIiwiaHR0cDovL2xvY2FsaG9zdDo1MDUwIiwiaHR0cDovL2xvY2FsaG9zdDo5OTQzIiwiaHR0cDovL2xvY2FsaG9zdDo5MDAwIl0sInJlYWxtX2FjY2VzcyI6eyJyb2xlcyI6WyJvZmZsaW5lX2FjY2VzcyIsImRlZmF1bHQtcm9sZXMtYWxpY2VibHVla2IiLCJ1bWFfYXV0aG9yaXphdGlvbiJdfSwicmVzb3VyY2VfYWNjZXNzIjp7ImFsaWNlLWtiIjp7InJvbGVzIjpbIkdVRVNUX1VTRVIiLCJBQ1RJVkVfVVNFUiJdfSwiYWNjb3VudCI6eyJyb2xlcyI6WyJtYW5hZ2UtYWNjb3VudCIsIm1hbmFnZS1hY2NvdW50LWxpbmtzIiwidmlldy1wcm9maWxlIl1

In [ ]:
import numpy as np
from scipy.signal import argrelextrema
import pandas as pd
from pya3 import Aliceblue
from datetime import datetime, timedelta
import time

def get_user_credentials():
    return "USER_ID", "API_KEY"  # Replace with actual credentials

def initialize_alice():
    user_id, api_key = get_user_credentials()
    alice = Aliceblue(user_id=user_id, api_key=api_key)
    print("Session ID:", alice.get_session_id())
    return alice

def compute_rsi(prices, window=14):
    delta = prices.diff().dropna()
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)

    avg_gain = gain.rolling(window).mean()
    avg_loss = loss.rolling(window).mean()

    rs = avg_gain / avg_loss
    return 100 - (100 / (1 + rs)).iloc[-1]

def find_swing_supports(df):
    closes = df['close'].values
    highs = df['high'].values
    lows = df['low'].values

    # Find swing lows using fractal detection
    swing_lows = []
    for i in range(2, len(lows)-2):
        if lows[i] < lows[i-1] and lows[i] < lows[i+1]:
            swing_lows.append(i)

    # Validate recent supports (last 30 days)
    valid_supports = []
    for idx in swing_lows:
        if idx < len(df)-30:
            continue
        support_price = lows[idx]
        current_price = closes[-1]

        # Look for 1-5% above support
        if 1.01 <= (current_price / support_price) <= 1.05:
            valid_supports.append({
                'price': support_price,
                'date': df.index[idx],
                'volume': df['volume'].iloc[idx]
            })

    return valid_supports

def analyze_swing_stock(alice, token):
    try:
        print(f"\nAnalyzing {token}...")
        instrument = alice.get_instrument_by_token('NSE', token)

        # Get 3 months data with 30min intervals
        historical_data = alice.get_historical(
            instrument,
            datetime.now() - timedelta(days=60),
            datetime.now(),
            "30"
        )
        df = pd.DataFrame(historical_data).dropna()

        if len(df) < 100:
            print("Insufficient data")
            return None

        # Basic indicators
        df['20EMA'] = df['close'].ewm(span=20).mean()
        current_rsi = compute_rsi(df['close'])

        # Find swing supports
        supports = find_swing_supports(df)
        if not supports:
            print("No valid supports")
            return None

        best_support = min(supports, key=lambda x: x['price'])
        current_price = df['close'].iloc[-1]

        # Entry conditions
        entry_conditions = (
            current_price > df['20EMA'].iloc[-1] and
            current_rsi < 60 and
            df['volume'].iloc[-1] > best_support['volume'] * 0.5 and
            (current_price / best_support['price']) <= 1.05
        )

        if not entry_conditions:
            print("Failed entry conditions")
            return None

        # Calculate risk/reward
        stop_loss = best_support['price'] * 0.985  # 1.5% below support
        risk = current_price - stop_loss
        reward = risk * 2  # 2:1 reward ratio
        target = current_price + reward

        return {
            'Token': token,
            'Name': instrument.name.split('-')[0].strip(),
            'Price': current_price,
            'Support': best_support['price'],
            'Stop Loss': stop_loss,
            'Target': target,
            'RSI': current_rsi,
            'Risk/Reward': f"{risk:.1f}/{reward:.1f}",
            'Volume': df['volume'].iloc[-1]
        }

    except Exception as e:
        print(f"Error analyzing {token}: {str(e)}")
        return None

def main():
    alice = initialize_alice()

    # Focus on liquid mid-cap stocks
    tokens = [
        11536,  # TCS
        1594,    # INFY
        10940,   # RELIANCE
        5900,    # BAJFINANCE
        3506,    # LT
        17818,   # HCLTECH
        5258,    # MARUTI
        3432,    # TITAN
        1333,    # SUNPHARMA
        694,     # HINDUNILVR
        236,     # ITC
        10604,   # ASIANPAINT
        2031,    # ONGC
        7229,    # ADANIENT
        11723,   # BAJAJFINSV
        16669,   # INDUSINDBK
        3045,    # DRREDDY
        3787,    # NESTLEIND
        13538,   # ULTRACEMCO
        1922     # WIPRO
    ]

    opportunities = []
    for token in tokens:
        result = analyze_swing_stock(alice, token)
        if result:
            opportunities.append(result)
        time.sleep(0.5)

    print("\nSwing Trading Opportunities:")
    print(f"{'Stock':<15} {'Price':<8} {'Support':<8} {'Stop Loss':<10} {'Target':<10} {'R/R':<10} {'RSI':<6}")
    for opp in opportunities:
        print(f"{opp['Name'][:12]:<15} {opp['Price']:<8.1f} {opp['Support']:<8.1f} "
              f"{opp['Stop Loss']:<10.1f} {opp['Target']:<10.1f} {opp['Risk/Reward']:<10} {opp['RSI']:<6.1f}")

if __name__ == "__main__":
    main()